In [1]:
import pandas as pd
import numpy as np

In [2]:
pip install openrouteservice

In [3]:
pip install folium

In [29]:
# === Import Libraries ===
import folium
import openrouteservice as ors

# === Coordinates in (latitude, longitude) format ===
coords = [
    [1.398472, 103.878123],
    [1.349812, 103.707451],
    [1.314929, 103.759623],
    [1.367810, 103.743922],
    [1.376220, 103.834600],
]

# === Vehicle starting point ===
vehicle_start = [1.352648, 103.831246]

# === Convert coordinates to ORS format (longitude, latitude) ===
converted_coords = [[lon, lat] for lat, lon in coords]
vehicle_start = [vehicle_start[1], vehicle_start[0]]  # Swap to (lon, lat)

# === Initialize ORS client ===
client = ors.Client(key='5b3')

# === Initialize vehicles ===
vehicles = [
    ors.optimization.Vehicle(id=0, profile='driving-car', start=vehicle_start, end=vehicle_start, capacity=[5]),
    ors.optimization.Vehicle(id=1, profile='driving-car', start=vehicle_start, end=vehicle_start, capacity=[5])
]

# === Initialize jobs (delivery points) ===
jobs = [ors.optimization.Job(id=index, location=loc, amount=[1]) for index, loc in enumerate(converted_coords)]

# === Optimize routes ===
optimized = client.optimization(jobs=jobs, vehicles=vehicles, geometry=True)

# === Initialize map ===
m = folium.Map(location=[vehicle_start[1], vehicle_start[0]], zoom_start=12)

# Add marker for each job location
for i, loc in enumerate(coords):
    folium.Marker(loc, popup=f"Job {i}").add_to(m)

# Add marker for vehicle home base
folium.Marker([vehicle_start[1], vehicle_start[0]], popup="Homebase", icon=folium.Icon(color='red')).add_to(m)

# === Define colors for vehicle routes ===
line_colors = ['green', 'orange', 'blue', 'yellow']

# === Add routes to the map ===
for route in optimized['routes']:
    route_coords = ors.convert.decode_polyline(route['geometry'])['coordinates']
    route_coords = [[lat, lon] for lon, lat in route_coords]  # Convert back to (lat, lon)
    folium.PolyLine(locations=route_coords, color=line_colors[route['vehicle']], weight=5, opacity=0.8).add_to(m)

# === Display route optimization results ===
for route in optimized['routes']:
    vehicle_id = route['vehicle']
    distance = route['distance']  # in meters
    duration = route['duration']  # in seconds
    duration_min = duration / 60

    print(f"\n🚗 Vehicle {vehicle_id + 1} ({line_colors[vehicle_id]} route):")

    # Extract job indices in the route steps
    job_indices = [step['job'] for step in route['steps'] if 'job' in step]

    # Print destination coordinates
    for idx in job_indices:
        print(f"  - {coords[idx]}")

    print(f"   - Total Distance: {distance / 1000:.2f} km")
    print(f"   - Total Duration: {duration_min:.2f} minutes\n")

print("✅ Route optimization completed!")

# === Display the map ===
m


🚗 Vehicle 1 (green route):
  - [1.314929, 103.759623]
  - [1.349812, 103.707451]
  - [1.36781, 103.743922]
  - [1.398472, 103.878123]
  - [1.37622, 103.8346]
   - Total Distance: 69.29 km
   - Total Duration: 106.33 minutes

✅ Route optimization completed!


In [30]:
import folium
import openrouteservice as ors

# Define coordinates with a service time for each destination:
# Format: [latitude, longitude, service_time_in_seconds]
destinations = [
    [1.398472, 103.878123, 600, 'critical'],
    [1.349812, 103.707451, 900, 'mid'],
    [1.314929, 103.759623, 1200, 'low'],
    [1.367810, 103.743922, 1800, 'high'],
    [1.376220, 103.834600, 1800, 'high'],
]


# Define vehicle homebase (latitude, longitude) -> ORS format (longitude, latitude)
vehicle_start = [1.352648, 103.831246]

# Convert the destination coordinates to ORS format (longitude, latitude) and extract service times.
converted_jobs = []
for dest in destinations:
    lat, lon, service_time, crit = dest
    converted_jobs.append({
        'location': [lon, lat],
        'service': service_time,
        'criticality': crit
    })

# Mapping for criticality levels to numeric priorities
priority_mapping = {
    "low": 20,
    "mid": 30,
    "high": 50,
    "critical": 100
}

# convert vehicle start coordinate to ORS format
vehicle_start = [vehicle_start[1], vehicle_start[0]]

# Initialize vehicle with a time_window constraint for a maximum of 8 hours (28,800 seconds)
vehicle = ors.optimization.Vehicle(
    id=0,
    profile='driving-car',
    start=vehicle_start,
    # end=vehicle_start,
    capacity=[len(destinations)],
    time_window=[0, 28800]  # Vehicle must complete its route within 8 hours
    )
# ]

# Patch the vehicle instance to include the custom costs parameter (set cost per km to 1000).
vehicle.__dict__["costs"] = {"per_km": 1000}

# Now vehicles is a list containing our patched vehicle instance.
vehicles = [vehicle]

# Define jobs using the converted coordinates, service times, and the priority (criticality)
jobs = [
    ors.optimization.Job(
        id=index,
        location=job['location'],
        amount=[1],
        service=job['service'],
        priority=priority_mapping[job['criticality']],
    )
    for index, job in enumerate(converted_jobs)
]

# Run route optimization
optimized = client.optimization(jobs=jobs, vehicles=vehicles, geometry=True)

# Define colors for vehicle routes (for terminal and map)
map_colors = ['green', 'orange', 'blue', 'yellow']

# Create a Folium map centered at the vehicle's start location (converted back to [latitude, longitude])
m = folium.Map(location=[vehicle_start[1], vehicle_start[0]], tiles="cartodbpositron", zoom_start=12)

# Add destination markers to the map
for dest in destinations:
    folium.Marker(location=dest[:2]).add_to(m)

# Mark the vehicle's start location in red
folium.Marker(location=[vehicle_start[1], vehicle_start[0]], icon=folium.Icon(color="red")).add_to(m)

# Add the optimized routes to the map and print route details
for route in optimized['routes']:
    vehicle_id = route['vehicle']
    route_color = map_colors[vehicle_id]

    # Decode the route geometry (polyline) and convert coordinates back to (latitude, longitude)
    route_coords = ors.convert.decode_polyline(route['geometry'])['coordinates']
    route_coords = [[lat, lon] for lon, lat in route_coords]

    # Add the route as a colored PolyLine on the map
    folium.PolyLine(locations=route_coords, color=route_color, weight=5).add_to(m)

    # Total available time for the vehicle in seconds (8 hours)
    total_time = 28800
    cumulative_time = 0
    prev_distance = 0  # Track cumulative distance from the start
    prev_duration = 0  # Track cumulative duration from the start


    # Print vehicle route information in the terminal
    print(f"\n🚗 Vehicle {vehicle_id + 1} ({route_color}):")

    # job_indices = [step['job'] for step in route['steps'] if 'job' in step]
    # for idx in job_indices:
    #     # Print out only the coordinates part (without service time) for clarity
    #     print(f"  {destinations[idx][:2]} (service: {destinations[idx][2]} sec)")

    for step in route['steps']:
      if 'job' in step:
        idx = step['job']

        # Calculate segment distance (current step's total - previous total)
        current_distance = step['distance']
        segment_distance = current_distance - prev_distance  # Distance from previous step
        prev_distance = current_distance  # Update previous distance

        # Calculate travel time (current step's total duration - previous total duration)
        current_duration = step['duration']
        travel_time = current_duration - prev_duration  # Travel time from previous step
        prev_duration = current_duration  # Update previous duration

        service_duration = destinations[idx][2]

        # Update cumulative time (travel + service)
        cumulative_time += travel_time + service_duration

        # Calculate remaining time
        remaining_time = total_time - cumulative_time
        remaining_hour = remaining_time / 3600

        # Convert to human-readable formats
        distance_km = segment_distance / 1000.0
        travel_time_minutes = travel_time / 60.0
        service_time_minutes = service_duration / 60.0

        # Compute cost for each destination using cost per km (1000)
        segment_cost = distance_km * 1000

        print(f"  {destinations[idx][:2]} | Distance: {distance_km:.2f} km, | "
              f"Cost: {segment_cost:.0f}, | "
              f"Priority: {destinations[idx][3]} | "
              f"Travel time: {travel_time_minutes:.1f} min, | "
              f"Service: {service_time_minutes:.1f} min, | "
              f"Remaining: {remaining_hour:.1f} H")

print("\n✅ Route optimization complete!")
m


🚗 Vehicle 1 (green):
  [1.37622, 103.8346] | Distance: 3.65 km, | Cost: 3649, | Priority: high | Travel time: 11.1 min, | Service: 30.0 min, | Remaining: 7.3 H
  [1.398472, 103.878123] | Distance: 9.44 km, | Cost: 9444, | Priority: critical | Travel time: 16.6 min, | Service: 10.0 min, | Remaining: 6.9 H
  [1.314929, 103.759623] | Distance: 29.10 km, | Cost: 29096, | Priority: low | Travel time: 29.1 min, | Service: 20.0 min, | Remaining: 6.1 H
  [1.349812, 103.707451] | Distance: 9.17 km, | Cost: 9171, | Priority: mid | Travel time: 15.6 min, | Service: 15.0 min, | Remaining: 5.5 H
  [1.36781, 103.743922] | Distance: 5.43 km, | Cost: 5429, | Priority: high | Travel time: 9.9 min, | Service: 30.0 min, | Remaining: 4.9 H

✅ Route optimization complete!
